# Maximum Representable Value

An exploratory notebook computing the maximum representable value for a given compression function.

More information is available on [ACEScentral](https://community.acescentral.com/t/rgb-saturation-gamut-mapping-approach-and-a-comp-vfx-perspective/2715/100?u=thomas_mansencal).

## colour-science Requirements

In [1]:
!pip install -q colour-science

!pip uninstall -y colour-science
!if ! [ -d "colour" ]; then git clone https://github.com/colour-science/colour; fi
!if [ -d "colour" ]; then cd colour && git fetch && git checkout develop && git pull && cd ..; fi

!python -c "import imageio;imageio.plugins.freeimage.download()"

Uninstalling colour-science-0.3.15:
  Successfully uninstalled colour-science-0.3.15
Already on 'develop'
Your branch is up to date with 'origin/develop'.
Already up to date.


In [2]:
from __future__ import division, unicode_literals

import sys

sys.path.append('colour')

import colour
import numpy as np

colour.plotting.colour_style()

colour.utilities.describe_environment();

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.6.9 (default, Apr 18 2020, 01:56:04)                       *
*                [GCC 8.4.0]                                                  *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-154-gfe631201                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.4.1                                                       *
*       matplotlib : 3.2.1                                                    *
*       networkx : 2.4                                                        *
*       numpy : 1.18.5                  

## Compression Functions

In [0]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, (a + b * np.tanh((x - a) / b)), x)


def atan_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + b * 2 / np.pi * np.arctan(((np.pi / 2) * (x - a)) / b), x)


def simple_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + (-1 / ((x - a) / b + 1) + 1) * b, x)


## Floating-Point Numbers in Domain [1-65504]

In [4]:
# !!! Float32 dtype will blow out Colab VM RAM !!!
DTYPE = np.float16
HFD = [1]

while True:
    HFD.append(np.nextafter(HFD[-1], +np.inf, dtype=DTYPE))
    if HFD[-1] > 65504:
        break

print(HFD)

[1, 1.0009766, 1.0019531, 1.0029297, 1.0039062, 1.0048828, 1.0058594, 1.0068359, 1.0078125, 1.0087891, 1.0097656, 1.0107422, 1.0117188, 1.0126953, 1.0136719, 1.0146484, 1.015625, 1.0166016, 1.0175781, 1.0185547, 1.0195312, 1.0205078, 1.0214844, 1.0224609, 1.0234375, 1.0244141, 1.0253906, 1.0263672, 1.0273438, 1.0283203, 1.0292969, 1.0302734, 1.03125, 1.0322266, 1.0332031, 1.0341797, 1.0351562, 1.0361328, 1.0371094, 1.0380859, 1.0390625, 1.0400391, 1.0410156, 1.0419922, 1.0429688, 1.0439453, 1.0449219, 1.0458984, 1.046875, 1.0478516, 1.0488281, 1.0498047, 1.0507812, 1.0517578, 1.0527344, 1.0537109, 1.0546875, 1.0556641, 1.0566406, 1.0576172, 1.0585938, 1.0595703, 1.0605469, 1.0615234, 1.0625, 1.0634766, 1.0644531, 1.0654297, 1.0664062, 1.0673828, 1.0683594, 1.0693359, 1.0703125, 1.0712891, 1.0722656, 1.0732422, 1.0742188, 1.0751953, 1.0761719, 1.0771484, 1.078125, 1.0791016, 1.0800781, 1.0810547, 1.0820312, 1.0830078, 1.0839844, 1.0849609, 1.0859375, 1.0869141, 1.0878906, 1.0888672, 1.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in nextafter
  """


## Computations of the Maximum Representable Value

The maximum representable value is computed as follows:

$argmax\big(f(HFD[x]) - f(HFD[x + 1])\big)$ where $HFD$ is the floating-point number array in domain [1, 65504] and $f$ is a given compression function.

In [5]:
for t in np.linspace(0, 1, 11):
    print(f'[ Threshold {np.around(t, 1)} ]')
    for func in (tanh_compression_function, atan_compression_function, simple_compression_function):
        print(func.__name__.split('_')[0],
              HFD[np.argmax((func(HFD[:-1], t, 1 - t) - func(HFD[1:], t, 1 - t)).astype(DTYPE))])
    print('\n')

[ Threshold 0.0 ]
tanh 6.58594
atan 7376.0
simple 23168.0


[ Threshold 0.1 ]
tanh 6.02734
atan 6636.0
simple 14744.0


[ Threshold 0.2 ]
tanh 5.46875
atan 5900.0
simple 13104.0


[ Threshold 0.3 ]
tanh 4.91016
atan 3650.0
simple 8108.0


[ Threshold 0.4 ]
tanh 4.35156
atan 3130.0
simple 6952.0


[ Threshold 0.5 ]
tanh 3.61914
atan 1844.0
simple 5792.0


[ Threshold 0.6 ]
tanh 3.0957
atan 1476.0
simple 3278.0


[ Threshold 0.7 ]
tanh 2.57227
atan 783.0
simple 1738.0


[ Threshold 0.8 ]
tanh 1.97852
atan 369.5
simple 820.0


[ Threshold 0.9 ]
tanh 1.48926
atan 93.0625
simple 205.625


[ Threshold 1.0 ]
tanh 1
atan 1
simple 1




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
